In [1]:
!pip install gcsfs

In [2]:
%%time
import pandas as pd
import os, sys, time, json, re, string, datetime

from pyspark import SparkContext, SparkConf, StorageLevel, keyword_only
from pyspark.sql.types import IntegerType

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.param.shared import HasInputCol, HasInputCols, HasOutputCol, HasOutputCols, Param
from pyspark.ml.feature import OneHotEncoder, HashingTF, IDF, Tokenizer, RegexTokenizer, NGram, CountVectorizer
from pyspark.ml.feature import StopWordsRemover, VectorAssembler, PCA, OneHotEncoderEstimator,StringIndexer

from pyspark.ml.classification import LogisticRegression, NaiveBayes, DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.classification import GBTClassifier, MultilayerPerceptronClassifier

from pyspark.ml import Pipeline, Transformer

from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from platform import python_version
print(python_version())

3.6.10
CPU times: user 56.2 ms, sys: 8.48 ms, total: 64.7 ms
Wall time: 62.3 ms


In [3]:
%%time
sc

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.68 µs


<SparkContext master=yarn appName=PySparkShell>

In [4]:
%%time
spark = SparkSession.builder \
        .appName("fakenews_baseline_model") \
        .config("spark.master", "yarn") \
        .config("spark.submit.deployMode", "cluster") \
        .config("spark.driver.memory", "24g") \
        .config("spark.executor.instances", "5") \
        .config("spark.executor.cores", "4") \
        .config("spark.executor.memory", "24g") \
        .getOrCreate()

CPU times: user 0 ns, sys: 7 ms, total: 7 ms
Wall time: 8.48 ms


In [5]:
%%time
df_fake_train = pd.read_csv('gs://dataproc-6ca41800-27b4-47d5-abee-55c011dfa389-asia-southeast1/data/fake-news/100k_fake_news_cleaned_dataset.csv')
df_fake_train[['authors_missing']] = df_fake_train[['authors_missing']].astype(int)
df_fake_train['label'] = 1
df_reliable_train = pd.read_csv('gs://dataproc-6ca41800-27b4-47d5-abee-55c011dfa389-asia-southeast1/data/fake-news/100k_reliable_news_cleaned_dataset.csv')
df_reliable_train[['authors_missing']] = df_reliable_train[['authors_missing']].astype(int)
df_reliable_train['label'] = 0
df_news_train = pd.concat([df_fake_train, df_reliable_train])
df_news_train[df_news_train.columns] = df_news_train[df_news_train.columns].astype(str)
print(df_news_train.info())
df_news_train

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217806 entries, 0 to 108010
Data columns (total 7 columns):
domain             217806 non-null object
type               217806 non-null object
content            217806 non-null object
title              217806 non-null object
authors            217806 non-null object
authors_missing    217806 non-null object
label              217806 non-null object
dtypes: object(7)
memory usage: 13.3+ MB
None
CPU times: user 8.84 s, sys: 2.12 s, total: 11 s
Wall time: 23 s


,domain,type,content,title,authors,authors_missing,label
0,beforeitsnews.com,fake,Boehner presses to make tax cuts permanent\n\n...,Boehner presses to make tax cuts permanent,United Liberty,0,1
1,beforeitsnews.com,fake,An Armed Good Samaritan Who Doesn’t Want The S...,An Armed Good Samaritan Who Doesn’t Want The S...,The Real Revo,0,1
2,beforeitsnews.com,fake,(Before It's News)\n\nby Rob Morphy\n\nLegends...,Village Of The Dead: The Anjikuni Mystery,"Rob Morphy, Mort Amsel",0,1
3,teaparty.org,fake,(Breitbart) – With his hysterical gotcha attac...,Trump Calls Out Race-Baiting ABC News Reporter,nan,1,1
4,beforeitsnews.com,fake,Researchers develop new depression diagnosis a...,Researchers develop new depression diagnosis a...,Bel Marra Health,0,1
...,...,...,...,...,...,...,...
108006,sports.yahoo.com,reliable,"View photos\nMichigan guard Zak Irvin, right, ...",No. 25 Michigan beats Mount St. Mary's 64-47,The Associated Press,0,0
108007,uk.finance.yahoo.com,reliable,President-elect Donald Trump continued his scr...,"Trump quotes Hillary Clinton, rages against Wi...",Maxwell Tani,0,0
108008,www.yahoo.com,reliable,Holiday shoppers eager to snag big discounts t...,Dialing up deals: Black Friday online sales hi...,ALEX VEIGA,0,0
108009,www.reuters.com,reliable,Kabul police raid shisha cafes in crackdown on...,Kabul police raid shisha cafes in crackdown on...,nan,1,0


In [6]:
%%time
df_news = spark.createDataFrame(df_news_train)
df_news = df_news.withColumn("label", df_news["label"].cast(IntegerType()))
df_news = df_news.withColumn("authors_missing", df_news["authors_missing"].cast(IntegerType()))
df_news.show(10)

+-----------------+----+--------------------+--------------------+--------------------+---------------+-----+
|           domain|type|             content|               title|             authors|authors_missing|label|
+-----------------+----+--------------------+--------------------+--------------------+---------------+-----+
|beforeitsnews.com|fake|Boehner presses t...|Boehner presses t...|      United Liberty|              0|    1|
|beforeitsnews.com|fake|An Armed Good Sam...|An Armed Good Sam...|       The Real Revo|              0|    1|
|beforeitsnews.com|fake|(Before It's News...|Village Of The De...|Rob Morphy, Mort ...|              0|    1|
|     teaparty.org|fake|(Breitbart) – Wit...|Trump Calls Out R...|                 nan|              1|    1|
|beforeitsnews.com|fake|Researchers devel...|Researchers devel...|    Bel Marra Health|              0|    1|
|beforeitsnews.com|fake|Trading Watch Lis...|Trading Watch Lis...|Bulls On Wall Street|              0|    1|
|beforeits

In [7]:
%%time
#remove empty content which will cause problem when transform the text
df_news = df_news.filter(df_news.content != "")
df_news = df_news.filter(df_news.title != "")
df_news = df_news.dropDuplicates(['label', 'content', 'title', 'authors_missing'])

CPU times: user 5.17 ms, sys: 1.01 ms, total: 6.18 ms
Wall time: 404 ms


In [8]:
# only keep type and content
df_news = df_news.select('label', 'content', 'title','authors_missing')

# split the dataset
df_train, df_test = df_news.randomSplit([0.8, 0.2], seed=666)
param_tuning = False

In [9]:
%%time
# customized transformer class to manually extract some counting based text features
class ReviewContentTransformer(Transformer, HasInputCol, HasOutputCol):

    @keyword_only
    def __init__(self, inputCol="content", outputCol="content_features"):
        super(ReviewContentTransformer, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)


    def _transform(self, dataset):
        
        def f(s):
            uppercase_count = 0
            char_count = 0
            for c in s:                
                if c in string.ascii_uppercase:
                    uppercase_count += 1
                    char_count += 1
                elif c in string.ascii_lowercase:
                    char_count += 1
            
            text_len = len(s)
            return Vectors.dense(text_len, char_count, 
                                 uppercase_count, uppercase_count / (char_count + 1e-10))

        return dataset.withColumn(self.getOutputCol(), 
                                  F.udf(f, VectorUDT())(dataset[self.getInputCol()]))

CPU times: user 94 µs, sys: 18 µs, total: 112 µs
Wall time: 119 µs


In [10]:
%%time
# customized transformer class to manually extract some counting based word features
class ReviewWordsTransformer(Transformer, HasInputCol, HasOutputCol):

    @keyword_only
    def __init__(self, inputCol="content", outputCol="content_features"):
        super(ReviewWordsTransformer, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)


    def _transform(self, dataset):
        
        def f(words):    
            word_count = len(words)
            unique_word_count = len(set(words))
            upper_words = []
            for w in words:
                if w.isupper():
                    upper_words.append(w)
            upper_word_count = len(set(upper_words))
            unique_upper_word_count = len(upper_words)
            return Vectors.dense(word_count, unique_word_count, unique_word_count / (word_count + 1e-10),
                                 upper_word_count, upper_word_count / (word_count + 1e-10), 
                                 unique_upper_word_count, unique_upper_word_count / (upper_word_count + 1e-10))

        return dataset.withColumn(self.getOutputCol(), 
                                  F.udf(f, VectorUDT())(dataset[self.getInputCol()]))

CPU times: user 100 µs, sys: 20 µs, total: 120 µs
Wall time: 126 µs


In [11]:
%%time
# show model prediction performance on the given dataset
def eval_model_perf(fitted_model, dataset, label_col="label", prediction_col="prediction", probability_col="probability"):
    pred_dataset = fitted_model.transform(dataset)
    eval_dataset = pred_dataset.select(label_col, prediction_col, probability_col)
    # model performance evaluation
    metricNames = ["accuracy", "f1"]
    model_eval = MulticlassClassificationEvaluator(predictionCol=prediction_col, labelCol=label_col)
    for m in metricNames:
        val = model_eval.evaluate(eval_dataset, {model_eval.metricName: m})
        print(m, " = ", val)
    roc_eval = BinaryClassificationEvaluator(rawPredictionCol=probability_col, labelCol=label_col, metricName="areaUnderROC")
    print("AUC =", roc_eval.evaluate(eval_dataset))    
    return pred_dataset

# show CV param tunning result
def show_cv_results(cv_model):
    for result, param in sorted(zip(cv_model.avgMetrics, cv_model.getEstimatorParamMaps()), reverse=True, key=lambda x: x[0]):
        print(result, " | ", param)

CPU times: user 12 µs, sys: 2 µs, total: 14 µs
Wall time: 18.6 µs


In [12]:
%%time

def run_models(df_train, df_test):
    print("**********LogisticRegression**********")
    t = time.time()
    lr_model = LogisticRegression(featuresCol='features', 
                                  labelCol='label', 
                                  predictionCol='prediction', 
                                  probabilityCol='probability', 
                                  rawPredictionCol='rawPrediction',
                                  family='binomial', 
                                  fitIntercept=True, 
                                  threshold=0.5, 
                                  standardization=False, 
                                  maxIter=200, 
                                  regParam=0.005, 
                                  elasticNetParam=0, 
                                  tol=1e-06, 
                                  aggregationDepth=2)

    lr_model = lr_model.fit(df_train)
    
    eval_model_perf(lr_model, df_test)
    
    print("time taken for LogisticRegression: ", str(datetime.timedelta(seconds=time.time() - t)))
    t = time.time()

    print("**********DecisionTreeClassifier**********")
    dt_model = DecisionTreeClassifier(featuresCol='features', 
                                      labelCol='label', 
                                      predictionCol='prediction', 
                                      probabilityCol='probability', 
                                      rawPredictionCol='rawPrediction', 
                                      maxDepth=10, maxBins=32, 
                                      minInstancesPerNode=1, 
                                      minInfoGain=0.0, 
                                      maxMemoryInMB=2048, 
                                      cacheNodeIds=True, 
                                      checkpointInterval=10,
                                      impurity='gini', 
                                      seed=666)

    dt_model = dt_model.fit(df_train)
    eval_model_perf(dt_model, df_test)
    print("time taken for DecisionTreeClassifier: ", str(datetime.timedelta(seconds=time.time() - t)))
    t = time.time()
    
    print("**********RandomForestClassifier**********")
    rf_model = RandomForestClassifier(featuresCol='features', 
                                      labelCol='label', 
                                      predictionCol='prediction', 
                                      probabilityCol='probability', 
                                      rawPredictionCol='rawPrediction',
                                      maxDepth=10, 
                                      maxBins=32, 
                                      minInstancesPerNode=1, 
                                      minInfoGain=0.0, 
                                      maxMemoryInMB=2048, 
                                      cacheNodeIds=True, 
                                      checkpointInterval=10, 
                                      impurity='gini', 
                                      numTrees=200, 
                                      featureSubsetStrategy='auto', 
                                      seed=666, 
                                      subsamplingRate=0.8)

    rf_model = rf_model.fit(df_train)
    eval_model_perf(rf_model, df_test)
    print("time taken for RandomForestClassifier: ", str(datetime.timedelta(seconds=time.time() - t)))
    t = time.time()
    
    print("**********GBTClassifier**********")
    gbt_model = GBTClassifier(featuresCol='features', 
                             labelCol='label', 
                             maxIter=250)

    gbt_model = gbt_model.fit(df_train)
    eval_model_perf(gbt_model, df_test)
    print("time taken for GBTClassifier: ", str(datetime.timedelta(seconds=time.time() - t)))
    t = time.time()    
    
    print("**********MultilayerPerceptronClassifier**********")
    mp_model = MultilayerPerceptronClassifier(featuresCol='features', 
                                              labelCol='label', 
                                              predictionCol='prediction', 
                                              layers=[4, 5, 4, 3],  
                                              maxIter=100, 
                                              blockSize=128, 
                                              seed=1234)

    mp_model = mp_model.fit(df_train)
    eval_model_perf(rf_model, df_test)
    print("time taken for MultilayerPerceptronClassifier: ", str(datetime.timedelta(seconds=time.time() - t)))
    t = time.time()  

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 11.9 µs


In [13]:
%%time
def build_data_preproc_model_with_pca(vocab_size=5000):
    preproc_steps = [
        RegexTokenizer(inputCol="content", outputCol="all_words", pattern=r"\W"),
        StopWordsRemover(inputCol="all_words", outputCol="words"),
        CountVectorizer(inputCol="words", outputCol="tf_features", vocabSize=vocab_size),
        IDF(inputCol="tf_features", outputCol="tfidf_features"),
        PCA(inputCol="tfidf_features", outputCol="pca_features", k=100),
        
        ReviewContentTransformer(inputCol="content", outputCol="content_features"),
        ReviewWordsTransformer(inputCol="words", outputCol="word_features"),
        
        RegexTokenizer(inputCol="title", outputCol="all_title_words", pattern=r"\W"),
        StopWordsRemover(inputCol="all_title_words", outputCol="title_words"),
        CountVectorizer(inputCol="title_words", outputCol="title_tf_features", vocabSize=100),
        IDF(inputCol="title_tf_features", outputCol="title_tfidf_features"),
        PCA(inputCol="title_tfidf_features", outputCol="title_pca_features", k=100),   
        
        StringIndexer(inputCol="authors_missing", outputCol="authors_missing_indexed", handleInvalid='keep'),
        OneHotEncoder(inputCol="authors_missing_indexed", outputCol="authors_missing_feature"),
        
        VectorAssembler(inputCols=["pca_features", "title_pca_features", "title_tfidf_features", 
                                   "content_features", "word_features", "authors_missing_feature"], 
                        outputCol="features")
    ]
    return Pipeline(stages=preproc_steps)

def build_data_preproc_model_without_pca(vocab_size=5000):
    preproc_steps = [
        RegexTokenizer(inputCol="content", outputCol="all_words", pattern=r"\W"),
        StopWordsRemover(inputCol="all_words", outputCol="words"),
        
        StringIndexer(inputCol="authors_missing", outputCol="authors_missing_indexed", handleInvalid='keep'),
        OneHotEncoder(inputCol="authors_missing_indexed", outputCol="authors_missing_feature"),
        
        ReviewContentTransformer(inputCol="content", outputCol="content_features"),
        ReviewWordsTransformer(inputCol="words", outputCol="word_features"),
        
        VectorAssembler(inputCols=["content_features", "word_features", "authors_missing_feature"], outputCol="features")
    ]
    return Pipeline(stages=preproc_steps)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 11 µs


In [ ]:
%%time
print("**********Run Models with PCA Features**********")
# generate the features to be used for model training
preproc_model = build_data_preproc_model_with_pca(2000).fit(df_train)
df_train_pca = preproc_model.transform(df_train).select("label", "features")
print(df_train_pca.take(1))
df_test_pca = preproc_model.transform(df_test).select("label", "features")
print(df_test_pca.take(1))
run_models(df_train_pca, df_test_pca)

**********Run Models with PCA Features**********
[Row(label=0, features=DenseVector([-18.8241, -3.4875, 1.7624, 4.4359, 3.5469, -5.2796, 1.0957, 1.7793, -5.1523, 2.0355, 0.7305, -4.1008, 1.4991, -2.0755, 1.9944, -4.8548, 1.0588, 5.061, -3.2008, -4.7146, 12.072, 4.5596, 0.6113, 2.2505, 1.8553, 3.8479, -6.1946, 2.9211, -8.9351, 1.606, -2.5601, -2.2386, -15.2543, 8.6931, 3.5452, -4.6838, 3.0176, 9.5615, -1.9669, -2.1753, -4.7353, 0.2118, -5.7341, -5.3679, -6.0063, 3.9125, 1.801, 9.1861, 2.8239, 1.2538, -1.4186, 7.1026, -1.094, -6.084, -1.8551, 5.504, 1.0316, -3.4852, -2.3373, -0.5905, -2.482, 1.5996, -8.4547, -1.1193, -5.0701, 7.5682, 4.9765, -1.8888, -0.4902, -5.4125, -0.1729, 1.7335, 4.8978, 2.9605, 3.0806, -2.9867, -4.0572, -6.3073, 3.4909, 6.2352, -1.0364, -1.0758, 3.2993, -0.6111, -2.1284, 3.8133, 4.0526, 5.2001, -5.7574, 0.2304, -0.2902, -0.5034, 6.2816, 3.214, 2.1842, -0.1246, -1.4028, -7.7809, -6.0749, -5.2691, -0.0254, 0.015, 0.0258, -0.0004, -0.0263, -0.0064, 0.028, -0.0451, 0.0

In [ ]:
%%time
print("**********Run Models without PCA Features**********")
# generate the features to be used for model training
preproc_model = build_data_preproc_model_without_pca(2000).fit(df_train)
df_train_wo_pca = preproc_model.transform(df_train).select("label", "features")
print(df_train_wo_pca.take(1))
df_test_wo_pca = preproc_model.transform(df_test).select("label", "features")
print(df_test_wo_pca.take(1))

nb_model = NaiveBayes(featuresCol='features', 
                      labelCol='label', 
                      predictionCol='prediction', 
                      probabilityCol='probability', 
                      rawPredictionCol='rawPrediction', 
                      smoothing=1, 
                      modelType='multinomial')

nb_model = nb_model.fit(df_train_wo_pca)
eval_model_perf(nb_model, df_test_wo_pca)

**********Run Models without PCA Features**********
[Row(label=0, features=DenseVector([5741.0, 4510.0, 182.0, 0.0404, 549.0, 329.0, 0.5993, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]))]
[Row(label=0, features=DenseVector([1862.0, 1481.0, 104.0, 0.0702, 182.0, 146.0, 0.8022, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]))]
accuracy  =  0.5702807536017732
f1  =  0.5659001574227993
AUC = 0.6287591998742492
CPU times: user 389 ms, sys: 184 ms, total: 573 ms
Wall time: 2min 11s


DataFrame[label: int, features: vector, rawPrediction: vector, probability: vector, prediction: double]